<a href="https://colab.research.google.com/github/carlavilla/DataViz2025/blob/main/CV_PS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exploring factors that may impact Philadelphia's PIT count
**Research questions:** <br>
- What variables within the current data collection methods of the PIT count appear to impact the accuracy of the count? <br>
- What variables outside of the data collection methods appear to impact the accuracy of the count? <br>

**Background:** <br>
[1](https://www.phila.gov/2022-12-15-the-2023-pit-count-is-coming-what-it-is-why-its-important-and-how-you-can-help/#:~:text=The%20U.S.%20Department%20of%20Housing,in%20January%20every%20other%20year.) The Point-In-Time (PIT) Count is a national count of homeless people across the country. The US Department of Housing and Urban Development (HUD) requires that state and local governments receiving federal funding to address homelessness, known as Continuums of Care or CoC, conduct an annual PIT Count in the month of January. <br>
2 The primary purpose of the PIT Count is to count homeless individuals, with a secondary purpose of collecting demographic information about homeless households (e.g. age, ethnicity, time homeless). There has been criticism about the accuracy of the PIT count. The survey tool is developed by HUD and deployed by each CoC. HUD does **not** require every counted individual to be surveyed. <br>
3  There have been criticisms about the accuracy of the PIT count. Most issues point to discrepancies across data collection methods and conditions outside of the protocol scope that may impact the number of individuals that get counted. Examples include number of counters, weather, hidden sheltering spaces (e.g. cars, abandoned structures), and criminalization of homelessness-adjacent activities.

# Merging data on homelessness count

##Environment set up

In [ ]:
#I was having trouble liking my online data last time and saw online a suggestion to use this code. I'm not sure if it's necessary. What does it do and when would it be necessary?
from google.colab import drive
drive.mount('/content/drive')
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

Mounted at /content/drive
Hello Google Drive!All changes made in this colab session should now be visible in Drive.


In [ ]:
#setting up basic libraries
import time, os, sys, re
import zipfile, json, datetime, string
import numpy as np
from statistics import *
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler

In [ ]:
#setting up link to online data and Colab
import urllib
from google.colab import files
import seaborn as sns
from google.colab import data_table
data_table.enable_dataframe_formatter()

##Retrieving Data Sets

###Dataset 1: PIT Count
**Description:** Overall count of homeless individuals at PIT counts in Philadelphia, available 2007 to 2023. <br>
**Relevance:** Dependent variable. Starting with overall count but will probably break it into sheltered and unshelterd count or focused on unsheltered. <br>
**Retrieved from:** Downloaded from [HUD PIT and HIC Data reservoir](https://www.hudexchange.info/resource/3031/pit-and-hic-data-since-2007/) on 2/5/2025.<br>
**Format:** .xlsb dataset that allows creation of a subtable. <br>
**Format for analyses:** Subtable was extracted for PA-500 CoC (Philadelphia) as an xlsx file. Data transposed on Excel.

In [ ]:
urllib.request.urlretrieve("https://drive.google.com/uc?id=1Rt9o407BcPjRLF2qzJljDHXqjLjWH0kj&export=download", "HUDPIT_PA500_2007to2023.xlsx")

('HUDPIT_PA500_2007to2023.xlsx', <http.client.HTTPMessage at 0x7a318ca36090>)

In [ ]:
pit = pd.read_excel('HUDPIT_PA500_2007to2023.xlsx', sheet_name= 'PA500_total')

In [ ]:
#checking headers and setting
pit.columns

Index(['year', 'total_count'], dtype='object')

In [ ]:
pit.head(2)

,year,total_count
0,2007,3869
1,2008,3479


In [ ]:
#Renaming for consistency and merging
pit = pit.rename(columns={'year': 'Year'})
pit.columns

Index(['Year', 'total_count'], dtype='object')

###Dataset 2: Temperature
**Description:** Mean for minimum temperature per month 2007-2023. <br>
**Relevance:** Independent variable. Lower temperatures may force homeless people to seek out last-resort informal shelter, reducing overall count. <br>
**Retrieved from:** Downloaded from the [National Weather Service](https://www.weather.gov/wrh/Climate?wfo=phi) on 2/12/2025.<br>
**Format:** data table generated within webpage, available for printing or download as pdf only. <br>
**Format for analyses:** Attempted to read pdf for analyses but could not figure out this time. Ended up converting the pdf to an .xlsx file on Adobe Acrobat. Cleaning of headers on Excel.

####First attempt: reading off pdf

In [ ]:
#installing tabula library
%pip install tabula-py
import tabula

In [ ]:
urllib.request.urlretrieve("https://drive.google.com/uc?id=14E0MRpMx77aHd1MEVPOb-r3eug-Mr4hM&export=download", "min_temp_2007to2023.pdf")

('min_temp_2007to2023.pdf', <http.client.HTTPMessage at 0x7a318c988590>)

In [ ]:
#QUESTION: command goes through but how do I see/manipulate the data?
from tabula import read_pdf
temp = read_pdf("min_temp_2007to2023.pdf", pages='all', lattice=True)

In [ ]:
#naturally, pandas commands do not work
#unclear how to proceed
#tabula documentation: https://pypi.org/project/tabula-py/
temp.head()

AttributeError: 'list' object has no attribute 'head'

In [ ]:
#tried converting the pdf to a csv
tabula.convert_into("min_temp_2007to2023.pdf", "min_temp_2007to2023.csv", output_format="csv", pages='all')

In [ ]:
#QUESTION: I think I need to give it the parameters for the dataframe but how do I apply the instructions?
# Tabula documentation https://pypi.org/project/tabula-py/
tempcsv = pd.read_csv('min_temp_2007to2023.csv')

In [ ]:
#QUESTION: why is the .csv file not recognized even though it is defined above?
tempcsv.head()

####Second attempt: using adobe to convert pdf to xlsx

In [ ]:
#retrieving .xlsx document converted on Adobe Acrobat, cleaned on Excel
urllib.request.urlretrieve("https://drive.google.com/uc?id=1myHP4UxOXomqz5JSAYJyo5L3R5v-KW_b&export=download", "min_temp_2007to2023.xlsx")
tempx=pd.read_excel('min_temp_2007to2023.xlsx')

In [ ]:
tempx.head(3)

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
0,2007,30.7,21.1,33.4,41.9,55.2,64.8,68.6,68.3,62.3,55.7,38.3,31.8,47.7
1,2008,28.6,28.3,35.7,46.8,51.4,66.7,70.4,65.6,62.2,47.6,39.1,30.8,47.8
2,2009,22.6,28.1,35.0,45.6,55.9,63.2,66.8,70.3,60.3,48.6,44.0,29.4,47.5


###Third Dataset: Demolitions

**Description:** Number of property demolitions initiated in Philadelphia 2007 to 2025. <br>
**Relevance:** Demolition of empty structures may result in fewer informal shelter opportunities for homeless folk, resulting in more visible homeless people during the count in January. <br>
**Retrieved from:** Downloaded from [Open Data Philly](https://opendataphilly.org/datasets/building-demolitions/) on 2/12/2025. <br>
**Format:** csv available on website for download <br>
**Format for analyses:** Demolitions are listed by address so there are multiple entries per year. Created a new sheet and used the Excel CountIf function to count demolitions for each calendar year. Since the demolitions for a given year (e.g. 2007) would hypothetically impact the count in January of the following year (e.g. 2008), a second column was created on Excel as "pit year" to indicate the year the data coulw be relevant for. Since .csv docs do now allow multiple sheets, the spreadsheet was saved as a .xlsx

In [ ]:
import pandas as pd
urllib.request.urlretrieve("https://drive.google.com/uc?id=1i_Uc-gNSdaHFVDcaCbX4o2CN04EvNEx7&export=download", "demolitions_2007to2025.xlsx")
demo=pd.read_excel('demolitions_2007to2025.xlsx', sheet_name='Sheet1')

In [ ]:
demo.head()

,year,year pit,Demolition count
0,2007,2008,315
1,2008,2009,667
2,2009,2010,548
3,2010,2011,758
4,2011,2012,803


In [ ]:
demo2 = demo.drop(columns=['year'])
demo2.columns

Index(['year pit', 'Demolition count'], dtype='object')

In [ ]:
#renaming year column to prep for merging
#QUESTION: Why do I have to create a new dataframe here whereas I didn't to do that for renaming it in the pit dataframe?
demo3=demo2.rename(columns={'year pit': 'Year'})
demo3.columns

Index(['Year', 'Demolition count'], dtype='object')

##Merging Datasets


###Merging Pit and Temp



In [ ]:
#Merge successful
pit_tempx = pd.merge(pit, tempx, how='outer', on=['Year'],indicator=True) #outer=keep all
pit_tempx

,Year,total_count,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual,_merge
0,2007,3869,30.7,21.1,33.4,41.9,55.2,64.8,68.6,68.3,62.3,55.7,38.3,31.8,47.7,both
1,2008,3479,28.6,28.3,35.7,46.8,51.4,66.7,70.4,65.6,62.2,47.6,39.1,30.8,47.8,both
2,2009,3054,22.6,28.1,35.0,45.6,55.9,63.2,66.8,70.3,60.3,48.6,44.0,29.4,47.5,both
3,2010,3077,26.2,25.7,39.1,47.6,58.0,68.8,72.6,70.4,63.7,50.3,40.2,26.4,49.1,both
4,2011,3328,23.1,28.3,35.5,47.2,57.8,66.1,72.8,67.8,63.7,49.2,42.4,35.2,49.1,both
5,2012,2976,29.9,33.1,42.7,45.4,59.2,63.7,72.7,70.4,62.3,52.0,36.5,36.5,50.4,both
6,2013,3070,29.4,28.5,33.4,44.9,55.4,65.6,73.1,67.3,58.5,52.2,36.9,31.7,48.1,both
7,2014,3109,19.4,24.9,29.6,43.5,56.0,65.3,69.2,66.1,62.2,52.2,37.2,35.1,46.7,both
8,2015,3346,23.9,17.5,30.5,45.4,59.3,66.5,71.3,70.0,65.4,50.0,44.7,44.2,49.1,both
9,2016,3430,26.1,30.8,41.2,44.5,54.8,64.7,72.5,72.4,64.8,51.7,41.1,32.1,49.7,both


In [ ]:
#creating sampled dataset of tempx to keep only Jan data as that is all I will use
#merge again with sampled dataset, successful
tempJan= tempx[['Year','Jan']]
pit_temp2 = pd.merge(pit, tempJan, how='outer', on=['Year'],indicator=True) #outer=keep all
pit_temp2

,Year,total_count,Jan,_merge
0,2007,3869,30.7,both
1,2008,3479,28.6,both
2,2009,3054,22.6,both
3,2010,3077,26.2,both
4,2011,3328,23.1,both
5,2012,2976,29.9,both
6,2013,3070,29.4,both
7,2014,3109,19.4,both
8,2015,3346,23.9,both
9,2016,3430,26.1,both


###Merging Pit, Temp, and Demo

In [ ]:
#Because I'm merging all of these sets based on year, it gave me an error because "Year" was a duplicate. AI suggested I change the indicator to something other than "True"
#Merge successful
#right only data makes sense because pit and temp datasets don't have data for those years
pit3 = pd.merge(pit_temp2, demo3, how='outer', on=['Year'],indicator='_merge2') #outer=keep all
pit3

,Year,total_count,Jan,_merge,Demolition count,_merge2
0,2007,3869.0,30.7,both,NaN,left_only
1,2008,3479.0,28.6,both,315.0,both
2,2009,3054.0,22.6,both,667.0,both
3,2010,3077.0,26.2,both,548.0,both
4,2011,3328.0,23.1,both,758.0,both
5,2012,2976.0,29.9,both,803.0,both
6,2013,3070.0,29.4,both,974.0,both
7,2014,3109.0,19.4,both,703.0,both
8,2015,3346.0,23.9,both,935.0,both
9,2016,3430.0,26.1,both,762.0,both


##Funsies Merge

###Dataset 4: Precipitation
**Description:** Mean precipitation for each month 2007-2024. <br>
**Relevance:** Independent variable. Higher precipitation may force homeless people to seek out last-resort informal shelter, reducing overall count. <br>
**Retrieved from:** Downloaded from the [National Weather Service](https://www.weather.gov/wrh/Climate?wfo=phi) on 2/12/2025.<br>
**Format:** data table generated within webpage, available for printing or download as pdf only. <br>
**Format for analyses:** Converted the pdf to an .xlsx file on Adobe Acrobat. Attempting cleaning here.

In [ ]:
#true headers at Index 1, first column not valid
urllib.request.urlretrieve("https://drive.google.com/uc?id=1vm5e50vTZJoyGvA75OtmboqdpHtNWtHA&export=download", "mean_precip_2007to2023.xlsx")
precip = pd.read_excel('mean_precip_2007to2023.xlsx')
precip.head(3)
#you can tell python where the header stars- find command to designate header

,"2/13/25, 12:49 AM about:blank",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,Monthly Mean Precipitation for Philadelphia A...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
2,NaN,2007,0.11,0.06,0.12,0.3,0.09,0.13,0.11,0.09,0.02,0.15,0.05,0.14,0.11


In [ ]:
#only need data for each year, do not need Index 20 to 24
precip.tail(7)

,"2/13/25, 12:49 AM about:blank",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
18,NaN,2023,0.11,0.05,0.06,0.17,0.01,0.14,0.17,0.1,0.2,0.02,0.09,0.26,0.12
19,NaN,2024,0.19,0.05,0.23,0.14,0.06,0.16,0.08,0.16,0.03,T,0.08,0.11,0.11
20,NaN,Mean,0.1,0.11,0.12,0.13,0.11,0.15,0.15,0.17,0.14,0.11,0.09,0.15,0.13
21,NaN,Max,0.19,0.22,0.24,0.3,0.21,0.35,0.43,0.62,0.34,0.19,0.3,0.29,0.18
22,NaN,NaN,2024,2018,2010,2007,2016,2013,2013,2011,2011,2022,2018,2009,NaN
23,NaN,Min,0.05,0.03,0.03,0.06,0.01,0.06,0.05,0.03,0.02,T,0.02,0.04,0.1
24,NaN,NaN,2021,2009,2012,2016,2023,2017,2012,2015,2007,2024,2021,2017,NaN


In [ ]:
#QUESTION: again, why did I have to create a new data frame here but not for pit?
#QUESTION: is there a way to refer to a column other than name? I wanted to rename that first column next to the index to "blank" but couldn't
precip2= precip.rename(columns={'Unnamed: 1': 'Year', 'Unnamed: 2': 'Jan'})
precip2.head()

,"2/13/25, 12:49 AM about:blank",Year,Jan,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,Monthly Mean Precipitation for Philadelphia A...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Annual
2,NaN,2007,0.11,0.06,0.12,0.3,0.09,0.13,0.11,0.09,0.02,0.15,0.05,0.14,0.11
3,NaN,2008,0.06,0.14,0.12,0.07,0.15,0.1,0.11,0.08,0.14,0.05,0.13,0.18,0.11
4,NaN,2009,0.09,0.03,0.05,0.13,0.16,0.16,0.11,0.33,0.12,0.18,0.07,0.29,0.14


In [ ]:
#removing irrelevant data
precip3= precip2[['Year', 'Jan']]
precip3.head()


,Year,Jan
0,Monthly Mean Precipitation for Philadelphia A...,NaN
1,Year,Jan
2,2007,0.11
3,2008,0.06
4,2009,0.09


In [ ]:
#removing extraneous rows
precip4 = precip3.drop(precip3.index[0:2])
precip4.head()

,Year,Jan
2,2007,0.11
3,2008,0.06
4,2009,0.09
5,2010,0.07
6,2011,0.11


In [ ]:
#checking index for tail
precip4.tail(7)

,Year,Jan
18,2023,0.11
19,2024,0.19
20,Mean,0.1
21,Max,0.19
22,NaN,2024
23,Min,0.05
24,NaN,2021


In [ ]:
#reseting index so I can delete the correct indeces
precip_reset= precip4.reset_index(drop=True)
precip_reset.tail(7)

,Year,Jan
16,2023,0.11
17,2024,0.19
18,Mean,0.1
19,Max,0.19
20,NaN,2024
21,Min,0.05
22,NaN,2021


In [ ]:
#dropping extraneous tail indices
precip5= precip_reset.drop(precip_reset.index[18:])
precip5.tail()

,Year,Jan
13,2020,0.09
14,2021,0.05
15,2022,0.11
16,2023,0.11
17,2024,0.19


In [ ]:
precip5.head()

,Year,Jan
0,2007,0.11
1,2008,0.06
2,2009,0.09
3,2010,0.07
4,2011,0.11


In [ ]:
#Merge successful
#left-only merges make sense
pit4 = pd.merge(pit3, precip5, how='outer', on=['Year'],indicator='_merge3')
pit4

,Year,total_count,Jan_x,_merge,Demolition count,_merge2,Jan_y,_merge3
0,2007,3869.0,30.7,both,NaN,left_only,0.11,both
1,2008,3479.0,28.6,both,315.0,both,0.06,both
2,2009,3054.0,22.6,both,667.0,both,0.09,both
3,2010,3077.0,26.2,both,548.0,both,0.07,both
4,2011,3328.0,23.1,both,758.0,both,0.11,both
5,2012,2976.0,29.9,both,803.0,both,0.08,both
6,2013,3070.0,29.4,both,974.0,both,0.11,both
7,2014,3109.0,19.4,both,703.0,both,0.11,both
8,2015,3346.0,23.9,both,935.0,both,0.15,both
9,2016,3430.0,26.1,both,762.0,both,0.08,both
